# Plotly and Streamlit

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/Plotly-logo.png/960px-Plotly-logo.png" height="150px" />

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/77/Streamlit-logo-primary-colormark-darktext.png/960px-Streamlit-logo-primary-colormark-darktext.png" height="150px" />

:::{dropdown} YouTube Video
:::{iframe} https://www.youtube.com/embed/vXpWp8P-fn8
:width: 100%
Video not loading? Click [here](https://www.youtube.com/watch?v=vXpWp8P-fn8).
:::

This workshop introduces you to interactive visualisation using the `plotly` library and the Streamlit framework for dashboards.

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Download the `.ipynb` file using the download button on the top right corner and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa atlite pandas geopandas xarray matplotlib geoviews plotly
%env HV_DOC_HTML=true
```
:::

In [ ]:
import pypsa
import atlite
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
from urllib.request import urlretrieve
from os.path import basename

urls = [
    "https://tubcloud.tu-berlin.de/s/2oogpgBfM5n4ssZ/download/PORTUGAL-2013-01-era5.nc",
]
for url in urls:
    urlretrieve(url, basename(url))

## Load Example Data

First, let's load a few example datasets you know from previous tutorials.

A PyPSA network:

In [ ]:
n = pypsa.Network(
    "https://tubcloud.tu-berlin.de/s/kpWaraGc9LeaxLK/download/network-cem.nc"
)

In [ ]:
n.optimize();

Wind, solar and demand time series:

In [ ]:
url = (
    "https://tubcloud.tu-berlin.de/s/nwCrNLrtL6LAN3W/download/time-series-lecture-2.csv"
)
ts = pd.read_csv(url, index_col=0, parse_dates=True)

Power plants in Europe

In [ ]:
url = (
    "https://raw.githubusercontent.com/PyPSA/powerplantmatching/v0.7.1/powerplants.csv"
)
ppl = pd.read_csv(url, index_col=0).query("Fueltype not in ['Wind', 'Solar']")

In [ ]:
geometry = gpd.points_from_xy(ppl["lon"], ppl["lat"])

In [ ]:
ppl = gpd.GeoDataFrame(ppl, geometry=geometry, crs=4326)

NUTS2 regions:

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/RHZJrN8Dnfn26nr/download/NUTS_RG_10M_2021_4326.geojson"
nuts = gpd.read_file(url)
nuts = nuts.set_index("id").query("LEVL_CODE == 2")

An `atlite` cutout:

In [ ]:
cutout = atlite.Cutout("PORTUGAL-2013-01-era5.nc")

## Limitations of Static Plots

You will agree that using `matplotlib` for static plotting is great for reports, but that it's lacking some features for interactive visualisation.

In [ ]:
ts["onwind [pu]"].plot(figsize=(10, 2))

## Interactive Plots with `plotly`

Specifically, we are going to use `plotly.express`, which is a high-level interface for `plotly`, to create interactive plots with just a few lines of code.

> The `plotly.express` module (usually imported as px) contains functions that can create entire figures at once. Plotly Express is a built-in part of the `plotly` library, and is the recommended starting point for creating most common figures. Every Plotly Express function uses graph objects internally and returns a plotly.graph_objects.Figure instance. Throughout the plotly documentation, you will find the Plotly Express way of building figures at the top of any applicable page, followed by a section on how to use graph objects to build similar figures. Any figure created in a single function call with Plotly Express could be created using graph objects alone, but with between 5 and 100 times more code.

:::{note}
Documentation is available at [plotly.com](https://plotly.com/python/plotly-express/)
:::

In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.offline as py

:::{note}
We need to import `plotly.io` and `plotly.offline`, so that the interactive plots are also visible on the course's static website.
:::

Let's create a few plots!

Onshore wind capacity factor time series:

In [ ]:
px.line(ts["onwind [pu]"])

Load time series in February:

In [ ]:
px.line(ts.loc["2015-02", "load [GW]"])

Scatter plot on map of hard coal power plants in Europe:

In [ ]:
df = ppl.query("Fueltype == 'Hard Coal'")

In [ ]:
px.scatter_mapbox(
    df, lat="lat", lon="lon", mapbox_style="carto-positron", zoom=2, height=600
)

In [ ]:
px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    mapbox_style="carto-positron",
    color="DateIn",
    size="Capacity",
    zoom=2,
    height=600,
)

Choropleth map of NUTS2 regions coloured by country:

In [ ]:
px.choropleth_mapbox(
    nuts,
    geojson=nuts.geometry,
    locations=nuts.index,
    mapbox_style="carto-positron",
    zoom=2,
    height=600,
    color="CNTR_CODE",
    center={"lat": 48, "lon": 12},
)

In `plotly`, hovering information can also be displayed well.

In [ ]:
dispatch = (
    pd.concat([n.generators_t.p, n.storage_units_t.p], axis=1).loc["2015-02"].div(1e3)
)

In [ ]:
df = dispatch.where(dispatch > 0, 0).stack().reset_index().rename(columns={0: "GW"})

In [ ]:
df.head(5)

In [ ]:
fig = px.area(df, x="snapshot", color="name", y="GW", line_group="name")
fig.update_traces(line=dict(width=0))
fig

## Interactive Dashboards

There are many different options for building interactive dashboards (e.g., Dash, Streamlit, Panel). Some are brand new, some have been around for a few years. Here, we are going to work with Streamlit as one example, since it is relatively easy to get started with and produce first results quickly. However, compared to other dashboarding frameworks, it has some limitations in terms of layout and interactivity.

:::{note}
Documentation is available at [streamlit.io](https://docs.streamlit.io/)
:::

In this tutorial, we look at `streamlit` because it is the easiest to get to results quickly. However, compared to other dashboarding libraries, it has more limited configuration options.

Documentation for this package can be found here: https://docs.streamlit.io/

Streamlit can be installed (e.g. with minimum version 1.18), for example, with `conda` or `pip`:

```sh
conda install -c conda-forge streamlit'>=1.18'
```

or

```sh
pip install streamlit">=1.18'
```

The rest of the tutorial is contained in a separate repository on Github with instructions how to install, run and deploy it:

https://github.com/fneum/streamlit-tutorial

You can see a live demo of the final product here:

https://ppm-dash.streamlit.app/
